In [1]:
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
import csv

overlay = Overlay("design_1_1.bit")


In [2]:
from pynq import allocate
from pynq import Overlay

WEIGHTS_01 = 12
WEIGHTS_12 = 4
WEIGHTS_23 = 2
WEIGHTS_34 = 2
WEIGHTS_45 = 2
WEIGHTS_56 = 4
WEIGHTS_67 = 12

dma = overlay.axi_dma_1

f = open("out.csv", "r")

weights = f.read().split(',')

for i in range(len(weights)):
    weights[i] = float(weights[i])

input_x = [1.0] * WEIGHTS_67

input_list = weights + input_x
print(input_list)

print(len(input_list))

[0.0, 0.0, 0.0, 0.0, -0.41245085, 0.22516125, 0.11589861, -0.24505067, 0.11002076, 0.35270548, 0.34254587, 0.25924808, 0.46233064, 0.09857297, 0.5462083, -0.47065502, 0.45651585, 0.22438496, 0.37261784, -0.2859464, 0.4596277, -0.15767288, 0.10052121, 0.56076676, -0.37306052, 0.22188687, 0.1600632, 0.56628186, -0.28270358, 0.19605273, -0.112050176, 0.54035777, 0.5763046, -0.07942265, -0.4533295, 0.34143978, -0.548317, -0.46473745, 0.12649375, 0.3921774, 0.037076235, 0.19918841, 0.4153989, -0.2784888, 0.19119453, 0.26479918, 0.48010808, -0.34708562, -0.29913658, -0.3335798, -0.24715337, -0.49087512, 0.0, 0.0, -0.7247691, 0.83424306, 0.6328485, -0.22648072, -0.6241925, -0.67744374, 0.06339741, -0.32233238, 0.0, 0.0, 0.8967639, -0.10948932, 0.52128124, -1.0032749, 0.0, 0.0, 0.7461281, 1.1210271, -0.29533046, 0.08058226, 0.0, 0.0, 0.0, 0.0, 0.7232716, 0.46492887, 0.13727546, -0.85849357, 0.9019425, 0.46246195, 0.14621854, 0.10324836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [3]:
input_buffer = allocate(shape=(158,), dtype=np.float32)
for i in range(158):
    input_buffer[i] = input_list[i]
    
res = allocate(shape=(1,), dtype=np.float32)

In [4]:
import time

start_time = time.time()

dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(res)
dma.sendchannel.wait()
dma.recvchannel.wait()

print("-----%s ms elapsed for transfer-----" %(1000*(time.time()-start_time)))

print(res)

-----1.3353824615478516 ms elapsed for transfer-----
[0.]


In [14]:
from pynq import Overlay
from pynq import allocate
import pynq.lib.dma
import numpy as np

class Model:
    def __init__(self, bitfile, paramfile):
        self.overlay = Overlay(bitfile)
        self.dma = self.overlay.axi_dma_1
        
        f = open(paramfile, "r")
        self.params = f.read().split(',')
        for i in range(len(self.params)):
            self.params[i] = float(self.params[i])
        self.numofparams = len(self.params)
        
        self.input_buffer = allocate(shape=(self.numofparams+12,), dtype=np.float32)
        self.res = allocate(shape=(1,), dtype=np.float32)
        
        
    def classify(self, input_x):
        for i in range(12):
            self.input_buffer[self.numofparams+i] = input_x[i]
        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.recvchannel.transfer(self.res)
        self.dma.sendchannel.wait()
        self.dma.recvchannel.wait()
        return int(self.res[0])
        

In [15]:
# Sample of using the Model class above
mlp = Model("design_1_1.bit", "out.csv")
# fake input here, shld be a list with 12 floats
input_x = [12.0] * 12
# classify function will return a class index (integer) with highest probability
mlp.classify(input_x)


0

-----1.3358592987060547 ms elapsed for transfer-----
[0.]


ModuleNotFoundError: No module named 'tensorflow'